In [1]:
'''
This is a script that will read in a few different csv files, as per the 
Udacity data analysis course instructions.
'''

import csv

# function to read in csv files, file's name as the arg
def read_csv(filename):
    # open the data file as a dictionary
    with open(filename, "rb") as f:
        reader = csv.DictReader(f)
        return list(reader)

# test the function on some csv files
#print(read_csv('enrollments.csv')[0])
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

In [2]:
for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]

In [3]:
enrollments[0]

{'account_key': '448',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_canceled': 'True',
 'is_udacity': 'True',
 'join_date': '2014-11-10',
 'status': 'canceled'}

In [4]:
daily_engagement[0]

{'account_key': '0',
 'lessons_completed': '0.0',
 'num_courses_visited': '1.0',
 'projects_completed': '0.0',
 'total_minutes_visited': '11.6793745',
 'utc_date': '2015-01-09'}

In [5]:
project_submissions[0]

{'account_key': '256',
 'assigned_rating': 'UNGRADED',
 'completion_date': '2015-01-16',
 'creation_date': '2015-01-14',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

In [6]:
enrollment_num_rows = len(enrollments)
print enrollment_num_rows

1640


In [7]:
def get_unique_students(data):
    unique_students = set()
    for key in data:
        unique_students.add(key['account_key'])
    return unique_students

In [8]:
unique_engagement_students = get_unique_students(daily_engagement)
unique_enrolled_students = get_unique_students(enrollments)
unique_project_submitters = get_unique_students(project_submissions)

In [9]:
for enrollment in enrollments: #loop over enrollment record
    student = enrollment['account_key'] # find account key for each enrollment
    if student not in unique_engagement_students: # check if account key is in the set
        print enrollment # if the student is issing, print the record
        break # break from the loop

{'status': 'canceled', 'is_udacity': 'False', 'is_canceled': 'True', 'join_date': '2014-11-12', 'account_key': '1219', 'cancel_date': '2014-11-12', 'days_to_cancel': '0'}


In [10]:
num_problem_students = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students \
        and enrollment['join_date'] != enrollment['cancel_date']:
            num_problem_students += 1
            print enrollment
            
num_problem_students

{'status': 'canceled', 'is_udacity': 'True', 'is_canceled': 'True', 'join_date': '2015-01-10', 'account_key': '1304', 'cancel_date': '2015-03-10', 'days_to_cancel': '59'}
{'status': 'canceled', 'is_udacity': 'True', 'is_canceled': 'True', 'join_date': '2015-03-10', 'account_key': '1304', 'cancel_date': '2015-06-17', 'days_to_cancel': '99'}
{'status': 'current', 'is_udacity': 'True', 'is_canceled': 'False', 'join_date': '2015-02-25', 'account_key': '1101', 'cancel_date': '', 'days_to_cancel': ''}


3

In [11]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity'] == "True":
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [12]:
def remove_udacity_accounts(data):
    non_udacity_account = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_account.append(data_point)
    return non_udacity_account

In [13]:
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagements = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagements)
print len(non_udacity_submissions)

1622
135656
3634


In [14]:
paid_students = {} # create empty dictionary for paid staudent accounts

for enrollment in non_udacity_enrollments: # loop over non udacity enrrolemnt records set
    if not enrollment['is_canceled'] or enrollment['days_to_cancel'] > 7: # want to add student to new dictionary if they haven't canceled yet, or if it was 7 days before they did cancel
        # pull out account key and enrollment date and save as vars
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
       
        if account_key not in paid_students or \
                enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date # add to the dictionary with the account key as the key and the enrollment date as the value
       
        
# how many students is this?
len(paid_students)

1296

In [15]:
# take the students join date and the date of a specific engagement record,
# and returns True  if that engagement  record  happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time.days < 7

In [16]:
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [17]:
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagements = remove_free_trial_cancels(non_udacity_engagements)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print len(paid_enrollments)
print len(paid_engagements)
print len(paid_submissions)

1622
135656
3626


In [18]:
paid_engagement_in_first_week = []

for engagement_record in paid_engagements:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']
    
    if within_one_week(join_date, engagement_record):
        paid_engagement_in_first_week.append(engagement_record)
        
len(paid_engagement_in_first_week)

TypeError: unsupported operand type(s) for -: 'dict' and 'str'